In [24]:
!git clone https://github.com/florvela/Object-Detection.git

Cloning into 'Object-Detection'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 70 (delta 10), reused 64 (delta 7), pack-reused 0
Unpacking objects: 100% (70/70), done.


In [25]:
%cd Object-Detection/Models/ssd

/content/Object-Detection/Models/ssd


In [26]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9BAcSz3Lo0NhgSxfp5gj")
project = rf.workspace("florencia").project("handgun-ogd8r")
dataset = project.version(1).download("coco")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Handgun-1 in coco:: 100%|██████████| 2201/2201 [00:01<00:00, 1534.87it/s]


In [27]:
%mv Handgun-1 datasets
%ls 

configs/        datasets/  train.py         utils/
custom_layers/  networks/  train_ssd.ipynb


In [30]:
%cd utils
!python3 prepare_COCO_dataset.py ../datasets/train/_annotations.coco.json ../datasets/train/ ../datasets/train/
!python3 prepare_COCO_dataset.py ../datasets/test/_annotations.coco.json ../datasets/test/ ../datasets/test/
!python3 prepare_COCO_dataset.py ../datasets/valid/_annotations.coco.json ../datasets/valid/ ../datasets/valid/

/content/Object-Detection/Models/ssd/utils
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
writing label maps to label_maps.txt
-- done
-- converting coco annotations to xml files
-- image 1/1920
-- image 2/1920
-- image 3/1920
-- image 4/1920
-- image 5/1920
-- image 6/1920
-- image 7/1920
-- image 8/1920
-- image 9/1920
-- image 10/1920
-- image 11/1920
-- image 12/1920
-- image 13/1920
-- image 14/1920
-- image 15/1920
-- image 16/1920
-- image 17/1920
-- image 18/1920
-- image 19/1920
-- image 20/1920
-- image 21/1920
-- image 22/1920
-- image 23/1920
-- image 24/1920
-- image 25/1920
-- image 26/1920
-- image 27/1920
-- image 28/1920
-- image 29/1920
-- image 30/1920
-- image 31/1920
-- image 32/1920
-- image 33/1920
-- image 34/1920
-- image 35/1920
-- image 36/1920
-- image 37/1920
-- image 38/1920
-- image 39/1920
-- image 40/1920
-- image 41/1920
-- image 42/1920
-- image 43/1920
-- image 44/1920
-- image 45/1920
-- image 46/1920
-- image 47/

In [31]:
%cd ..
import os
import json
from tensorflow.keras.optimizers import SGD, Adam
from utils import ModelCheckpoint
from utils import training_utils

args = {
    'config': './configs/vgg16_flor.json',
    'images_dir': './datasets/',
    'labels_dir': './datasets/',
    'training_split': './datasets/train/split.txt',
    'validation_split':  './datasets/valid/split.txt',
    'label_maps': ['Handgun'],
    'checkpoint': None , # can be an existing h5 to load weights from and continue training
    'checkpoint_type': 'epoch',
    'checkpoint_frequency': 10,
    'initial_epoch': 0,
    'learning_rate': 0.001, #de aca https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
    'epochs': 20,
    'batch_size': 32, # https://stats.stackexchange.com/questions/164876/what-is-the-trade-off-between-batch-size-and-number-of-iterations-to-train-a-neu
    'shuffle': True,
    'augment': True,
    'output_dir': 'output'# '/content/drive/MyDrive/ssd_train_output_64_batches_001_lr_augmented_adam'
}

assert os.path.exists(args["config"]), "config file does not exist"
assert os.path.exists(args["images_dir"]), "images_dir does not exist"
assert os.path.exists(args["labels_dir"]), "labels_dir does not exist"
assert args["epochs"] > 0, "epochs must be larger than zero"
assert args["batch_size"] > 0, "batch_size must be larger than 0"
assert args["learning_rate"] > 0, "learning_rate must be larger than 0"
assert args["checkpoint_type"] in ["epoch", "iteration", "none"], "checkpoint_type must be one of epoch, iteration, none."

if not os.path.exists(args["output_dir"]):
    os.makedirs(args["output_dir"])

with open(args["config"], "r") as config_file:
    config = json.load(config_file)

training_data_generator, num_training_samples, validation_data_generator, num_validation_samples = training_utils.get_data_generator(config, args)
model = training_utils.get_model(config, args["label_maps"])
loss = training_utils.get_loss(config)
# optimizer = SGD(
#             learning_rate=args["learning_rate"],
#             momentum=0.9,
#             decay=0.0005,
#             nesterov=False
#         )

optimizer = Adam(
    learning_rate=args["learning_rate"],
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False
)

model.compile(optimizer=optimizer, loss=loss.compute)


num_iterations_per_epoch = num_training_samples//args["batch_size"]
if args["checkpoint_type"] == "epoch":
    assert args["checkpoint_frequency"] < args["epochs"], "checkpoint_frequency must be smaller than epochs."
elif args["checkpoint_type"] == "iteration":
    assert args["checkpoint_frequency"] < num_iterations_per_epoch * args["epochs"], "checkpoint_frequency must be smaller than num_iterations_per_epoch * args_epochs"

if args["checkpoint"] is not None:
    assert os.path.exists(args["checkpoint"]), "checkpoint does not exist"
    model.load_weights(args["checkpoint"], by_name=True)

model.fit(
    x=training_data_generator,
    validation_data=validation_data_generator,
    batch_size=args["batch_size"],
    validation_batch_size=args["batch_size"],
    epochs=args["epochs"],
    initial_epoch = args["initial_epoch"],
    callbacks=[
        ModelCheckpoint(
            initial_epoch = args["initial_epoch"],
            output_dir=args["output_dir"],
            epoch_frequency=args["checkpoint_frequency"] if args["checkpoint_type"] == "epoch" else None,
            iteration_frequency=args["checkpoint_frequency"] if args["checkpoint_type"] == "iteration" else None,
        )
    ]
)



/content/Object-Detection/Models/ssd
creating data generator for ssd_vgg16
-- validation split specified
58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 zero_padding2d (ZeroPadding  (None, 304, 304, 3)      0         
 2D)                                                             
                                                                 
 block1_conv1 (Conv2D)       (None, 304, 304, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 304, 304, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 152, 152, 64)      0         
                       

KeyboardInterrupt: ignored